<a href="https://colab.research.google.com/github/PetitYohan/DataMining/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projet de DataMining
Par Catherine Vanden Hende & Yohan Petit

##Initialisation du Projet

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600~/.kaggle/kaggle.json
! kaggle datasets download -d razinw/dog-vs-cat
! mkdir images
! unzip dog-vs-cat.zip -d images/

##Récupération des métadonnées dans des json

In [21]:
!pip3 install Pillow
!mkdir metadata
!mkdir metadata/train
!mkdir metadata/train/0
!mkdir metadata/train/1

from PIL import Image
from PIL.ExifTags import TAGS
import os

# path to the directory
path = "images/dogvscat/train/0"
# directory
directory = os.listdir(path)

for files in directory:
  if files.endswith ('.jpg'):
    # find file path
    file_path = os.path.join(path,files)
    # read the image data using PIL
    image = Image.open(file_path)
    # extract EXIF data
    exifdata = image.getexif()
    # iterating over all EXIF data fields
    for tag_id in exifdata:
        # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id, tag_id)
        data = exifdata.get(tag_id)
        # decode bytes 
        if isinstance(data, bytes):
            data = data.decode()
        print(f"{tag:25}: {data}")
    print("\n")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
ExifImageHeight          : 1000
ImageWidth               : 3648
ImageLength              : 2736
Flash                    : (24, 0)
FocalLength              : (89, 1)
BitsPerSample            : (8, 8, 8)
PhotometricInterpretation: 2
Make                     : OLYMPUS IMAGING CORP.
Model                    : E-450
Orientation              : 1
SamplesPerPixel          : 3
ExposureTime             : (1, 160)
XResolution              : (3000000, 10000)
YResolution              : (3000000, 10000)
FNumber                  : (41, 10)
ISOSpeedRatings          : (400, 0)
ResolutionUnit           : 2
ExifOffset               : 308
Software                 : Adobe Photoshop CC 2019 (Windows)
DateTime                 : 2019:03:04 21:44:37
LensModel                : OLYMPUS 70mm-300mm Lens


ExifVersion              : 0221
ShutterSpeedValue        : (8965784, 1000000)
ApertureValue            : (5655638, 1000000)
DateTimeO